## Description of Notebook

The notebook is used to test ML approach on the VDF statistical momenta, anisotropies, and particle fraction numbers. Here we also vary the labels based on the threshold (five categories of labels were defined during the data processing phase)

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN

In [2]:
def outputclass_analysis(test_labels, predicted_labels, output_score=''):
    tn, fp, fn, tp = confusion_matrix(test_labels, predicted_labels).ravel()
    print("------------ SUMMARY OF CLASSIFICATION RESULTS ----------------")
    print("TP = "+str(tp))
    print("TN = "+str(tn))
    print("FP = "+str(fp))
    print("FN = "+str(fn))
    precision = tp/(tp+fp) if ((tp+fp) != 0) else -999.9
    recall = tp/(tp+fn) if ((tp+fn) != 0) else -999.9
    acc = (tp+tn)/(tp+fn+fp+tn)
    tss = tp/(tp+fn) - fp/(fp+tn) if (((tp+fn) != 0) and ((fp+tn) != 0)) else -999.9
    hss = 2*(tp*tn - fp*fn)/((tp+fn)*(fn+tn) + (tp+fp)*(fp+tn))
    print("Precision = "+str(precision))
    print("Recall = "+str(recall))
    print("Accuracy = "+str(acc))
    print("TSS = "+str(tss))
    print("HSS = "+str(hss))
    if (output_score == 'TSS'): return tss
    if (output_score == 'HSS'): return hss
    if (output_score == 'precision'): return precision
    if (output_score == 'accuracy'): return acc
    return

def outputclass_analysis_scorereturn(test_labels, predicted_labels):
    matrix_elements = confusion_matrix(test_labels, predicted_labels, labels=[0, 1]).ravel()
    tn, fp, fn, tp = matrix_elements
    precision = tp/(tp+fp) if ((tp+fp) != 0) else -999.9
    recall = tp/(tp+fn) if ((tp+fn) != 0) else -999.9
    acc = (tp+tn)/(tp+fn+fp+tn)
    tss = tp/(tp+fn) - fp/(fp+tn) if (((tp+fn) != 0) and ((fp+tn) != 0)) else -999.9
    return tp, tn, fp, fn, acc, tss

## Test 1. Preparation and testing of the data set.

Various thresholds are tried here.

In [3]:
def label_union(indstr):
    labels_an = np.load('./mldata_vdfmoments/allsimulations.labels_allmoments_an_'+indstr+'_all.npy')
    labels_me = np.load('./mldata_vdfmoments/allsimulations.labels_allmoments_me_'+indstr+'_all.npy')
    labels_allmoments = np.copy(labels_me)
    labels_allmoments[np.where(labels_an == 1)] = 1
    print('------------------------- THRES = ' + indstr + '----------------------------------')
    print('The total number of data points is: ' + str(len(labels_allmoments)))
    print('Among them unstable (positive) samples: ' + str(len(np.where(labels_allmoments == 1)[0])))
    return labels_allmoments

featurevector_allmoments = np.load('./mldata_vdfmoments/allsimulations.featurevector_allmoments_all.npy')
times_allmoments = np.load('./mldata_vdfmoments/allsimulations.timep_array_all.npy')
labels_allmoments_001 = label_union('001')
labels_allmoments_005 = label_union('005')
labels_allmoments_01 = label_union('01')
labels_allmoments_05 = label_union('05')
labels_allmoments_10 = label_union('10')

------------------------- THRES = 001----------------------------------
The total number of data points is: 1596
Among them unstable (positive) samples: 1363
------------------------- THRES = 005----------------------------------
The total number of data points is: 1596
Among them unstable (positive) samples: 681
------------------------- THRES = 01----------------------------------
The total number of data points is: 1596
Among them unstable (positive) samples: 418
------------------------- THRES = 05----------------------------------
The total number of data points is: 1596
Among them unstable (positive) samples: 110
------------------------- THRES = 10----------------------------------
The total number of data points is: 1596
Among them unstable (positive) samples: 62


In [4]:
scaler = StandardScaler()
scaler.fit(featurevector_allmoments)
featurevector_allmoments = scaler.transform(featurevector_allmoments)

In [5]:
data_split = ShuffleSplit(n_splits=10, test_size=0.33, random_state=0)

## Test 2. Random Forest on the data sets of various thresholds

Given that Random Forest has demonstrated a good performance previously, we down-select it for the thresholds.

In [6]:
def evaluate_RFclassifier(featurevector_allmoments, labels_allmoments, data_split):

    # parameter grid
    param_grid = {'n_estimators': [10,50,100,200], \
                  'max_depth': [None, 2, 5, 10, 25], \
                  'class_weight': [None, 'balanced']}

    # classifier and gridsearch
    clf = RandomForestClassifier()
    grid_search = GridSearchCV(clf, param_grid, cv=data_split, verbose=1, scoring='accuracy')
    grid_search.fit(featurevector_allmoments, labels_allmoments)
    print("Best parameters found: ", grid_search.best_params_)
    best_params = grid_search.best_params_

    # fitting 10 times and accumulating the scores for the best model
    tp = np.zeros([10], dtype=int)
    tn = np.zeros([10], dtype=int)
    fp = np.zeros([10], dtype=int)
    fn = np.zeros([10], dtype=int)
    acc = np.zeros([10], dtype=float)
    tss = np.zeros([10], dtype=float)

    for i, split_indexes in enumerate(data_split.split(featurevector_allmoments)):
        train_index, test_index = split_indexes
        X_train, X_test = featurevector_allmoments[train_index], featurevector_allmoments[test_index]
        f_train, f_test = labels_allmoments[train_index], labels_allmoments[test_index]
        clf = RandomForestClassifier(**best_params)
        clf.fit(X_train, f_train)
        f_predicted = clf.predict(X_test)
        tp[i], tn[i], fp[i], fn[i], acc[i], tss[i] = outputclass_analysis_scorereturn(f_test, f_predicted)

    print("TP = " + str(np.mean(tp)) + "+/-" + str(np.std(tp)))
    print("TN = " + str(np.mean(tn)) + "+/-" + str(np.std(tn)))
    print("FP = " + str(np.mean(fp)) + "+/-" + str(np.std(fp)))
    print("FN = " + str(np.mean(fn)) + "+/-" + str(np.std(fn)))
    print("Acc = " + str(np.mean(acc)) + "+/-" + str(np.std(acc)))
    print("TSS = " + str(np.mean(tss)) + "+/-" + str(np.std(tss)))
    

print("-------------------------------------")
print("Testing for threshold THRES = 0.0001:")
evaluate_RFclassifier(featurevector_allmoments, labels_allmoments_001, data_split)

print("-------------------------------------")
print("Testing for threshold THRES = 0.0005:")
evaluate_RFclassifier(featurevector_allmoments, labels_allmoments_005, data_split)

print("-------------------------------------")
print("Testing for threshold THRES = 0.001:")
evaluate_RFclassifier(featurevector_allmoments, labels_allmoments_01, data_split)

print("-------------------------------------")
print("Testing for threshold THRES = 0.005:")
evaluate_RFclassifier(featurevector_allmoments, labels_allmoments_05, data_split)

print("-------------------------------------")
print("Testing for threshold THRES = 0.01:")
evaluate_RFclassifier(featurevector_allmoments, labels_allmoments_10, data_split)

-------------------------------------
Testing for threshold THRES = 0.0001:
Fitting 10 folds for each of 40 candidates, totalling 400 fits
Best parameters found:  {'class_weight': 'balanced', 'max_depth': 10, 'n_estimators': 100}
TP = 435.3+/-6.664082832618455
TN = 71.3+/-5.367494760127857
FP = 8.0+/-3.0
FN = 12.4+/-2.33238075793812
Acc = 0.9612903225806452+/-0.003907260015554844
TSS = 0.8733232940514501+/-0.028799332646232665
-------------------------------------
Testing for threshold THRES = 0.0005:
Fitting 10 folds for each of 40 candidates, totalling 400 fits
Best parameters found:  {'class_weight': None, 'max_depth': None, 'n_estimators': 50}
TP = 206.6+/-5.351635264103861
TN = 293.7+/-5.478138369920935
FP = 11.0+/-3.9496835316262997
FN = 15.7+/-5.254521862167861
Acc = 0.9493358633776092+/-0.00906258363283457
TSS = 0.8938909543926948+/-0.019903829633524488
-------------------------------------
Testing for threshold THRES = 0.001:
Fitting 10 folds for each of 40 candidates, totalli

Same as above but with the restricted class weight set to balance

In [7]:
def tss_score(test_labels, predicted_labels):
    matrix_elements = confusion_matrix(test_labels, predicted_labels, labels=[0, 1]).ravel()
    tn, fp, fn, tp = matrix_elements
    tss = tp/(tp+fn) - fp/(fp+tn) if (((tp+fn) != 0) and ((fp+tn) != 0)) else -999.9
    return tss


def evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments, data_split, tss_scorer):

    # parameter grid
    param_grid = {'n_estimators': [10,50,100,200], \
                  'max_depth': [None, 2, 5, 10, 25], \
                  'class_weight': [None, 'balanced']}

    # classifier and gridsearch
    clf = RandomForestClassifier()
    grid_search = GridSearchCV(clf, param_grid, cv=data_split, verbose=1, scoring=tss_scorer)
    grid_search.fit(featurevector_allmoments, labels_allmoments)
    print("Best parameters found: ", grid_search.best_params_)
    best_params = grid_search.best_params_

    # fitting 10 times and accumulating the scores for the best model
    tp = np.zeros([10], dtype=int)
    tn = np.zeros([10], dtype=int)
    fp = np.zeros([10], dtype=int)
    fn = np.zeros([10], dtype=int)
    acc = np.zeros([10], dtype=float)
    tss = np.zeros([10], dtype=float)

    for i, split_indexes in enumerate(data_split.split(featurevector_allmoments)):
        train_index, test_index = split_indexes
        X_train, X_test = featurevector_allmoments[train_index], featurevector_allmoments[test_index]
        f_train, f_test = labels_allmoments[train_index], labels_allmoments[test_index]
        clf = RandomForestClassifier(**best_params)
        clf.fit(X_train, f_train)
        f_predicted = clf.predict(X_test)
        tp[i], tn[i], fp[i], fn[i], acc[i], tss[i] = outputclass_analysis_scorereturn(f_test, f_predicted)

    print("TP = " + str(np.mean(tp)) + "+/-" + str(np.std(tp)))
    print("TN = " + str(np.mean(tn)) + "+/-" + str(np.std(tn)))
    print("FP = " + str(np.mean(fp)) + "+/-" + str(np.std(fp)))
    print("FN = " + str(np.mean(fn)) + "+/-" + str(np.std(fn)))
    print("Acc = " + str(np.mean(acc)) + "+/-" + str(np.std(acc)))
    print("TSS = " + str(np.mean(tss)) + "+/-" + str(np.std(tss)))
    

tss_scorer = make_scorer(tss_score, greater_is_better=True)
    
print("-------------------------------------")
print("Testing for threshold THRES = 0.0001:")
evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments_001, data_split, tss_scorer)

print("-------------------------------------")
print("Testing for threshold THRES = 0.0005:")
evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments_005, data_split, tss_scorer)

print("-------------------------------------")
print("Testing for threshold THRES = 0.001:")
evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments_01, data_split, tss_scorer)

print("-------------------------------------")
print("Testing for threshold THRES = 0.005:")
evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments_05, data_split, tss_scorer)

print("-------------------------------------")
print("Testing for threshold THRES = 0.01:")
evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments_10, data_split, tss_scorer)

-------------------------------------
Testing for threshold THRES = 0.0001:
Fitting 10 folds for each of 40 candidates, totalling 400 fits
Best parameters found:  {'class_weight': 'balanced', 'max_depth': 5, 'n_estimators': 10}
TP = 417.4+/-8.777243302996675
TN = 75.1+/-6.847627326307997
FP = 4.2+/-1.4696938456699067
FN = 30.3+/-6.649060083951716
Acc = 0.9345351043643264+/-0.012068370866153968
TSS = 0.8797083264868661+/-0.01935982146633678
-------------------------------------
Testing for threshold THRES = 0.0005:
Fitting 10 folds for each of 40 candidates, totalling 400 fits
Best parameters found:  {'class_weight': None, 'max_depth': None, 'n_estimators': 50}
TP = 206.8+/-6.321392251711643
TN = 293.9+/-6.268173577685927
FP = 10.8+/-4.621688003316537
FN = 15.5+/-4.631414470763764
Acc = 0.9500948766603416+/-0.009677419354838703
TSS = 0.8952542196251961+/-0.020106730224330627
-------------------------------------
Testing for threshold THRES = 0.001:
Fitting 10 folds for each of 40 candid

## Test 3. Same as above but for the data with power spectrum.

In [8]:
def label_union(indstr):
    labels_an = np.load('./mldata_vdfmoments/allsimulations.labels_allmoments_an_'+indstr+'_allps.npy')
    labels_me = np.load('./mldata_vdfmoments/allsimulations.labels_allmoments_me_'+indstr+'_allps.npy')
    labels_allmoments = np.copy(labels_me)
    labels_allmoments[np.where(labels_an == 1)] = 1
    print('------------------------- THRES = ' + indstr + '----------------------------------')
    print('The total number of data points is: ' + str(len(labels_allmoments)))
    print('Among them unstable (positive) samples: ' + str(len(np.where(labels_allmoments == 1)[0])))
    return labels_allmoments

featurevector_allmoments = np.load('./mldata_vdfmoments/allsimulations.featurevector_allmoments_allps.npy')
times_allmoments = np.load('./mldata_vdfmoments/allsimulations.timep_array_allps.npy')
labels_allmoments_001 = label_union('001')
labels_allmoments_005 = label_union('005')
labels_allmoments_01 = label_union('01')
labels_allmoments_05 = label_union('05')
labels_allmoments_10 = label_union('10')

------------------------- THRES = 001----------------------------------
The total number of data points is: 1596
Among them unstable (positive) samples: 1363
------------------------- THRES = 005----------------------------------
The total number of data points is: 1596
Among them unstable (positive) samples: 681
------------------------- THRES = 01----------------------------------
The total number of data points is: 1596
Among them unstable (positive) samples: 418
------------------------- THRES = 05----------------------------------
The total number of data points is: 1596
Among them unstable (positive) samples: 110
------------------------- THRES = 10----------------------------------
The total number of data points is: 1596
Among them unstable (positive) samples: 62


In [9]:
scaler = StandardScaler()
scaler.fit(featurevector_allmoments)
featurevector_allmoments = scaler.transform(featurevector_allmoments)

In [10]:
data_split = ShuffleSplit(n_splits=10, test_size=0.33, random_state=0)

In [11]:
def evaluate_RFclassifier(featurevector_allmoments, labels_allmoments, data_split):

    # parameter grid
    param_grid = {'n_estimators': [10,50,100,200], \
                  'max_depth': [None, 2, 5, 10, 25], \
                  'class_weight': [None, 'balanced']}

    # classifier and gridsearch
    clf = RandomForestClassifier()
    grid_search = GridSearchCV(clf, param_grid, cv=data_split, verbose=1, scoring='accuracy')
    grid_search.fit(featurevector_allmoments, labels_allmoments)
    print("Best parameters found: ", grid_search.best_params_)
    best_params = grid_search.best_params_

    # fitting 10 times and accumulating the scores for the best model
    tp = np.zeros([10], dtype=int)
    tn = np.zeros([10], dtype=int)
    fp = np.zeros([10], dtype=int)
    fn = np.zeros([10], dtype=int)
    acc = np.zeros([10], dtype=float)
    tss = np.zeros([10], dtype=float)

    for i, split_indexes in enumerate(data_split.split(featurevector_allmoments)):
        train_index, test_index = split_indexes
        X_train, X_test = featurevector_allmoments[train_index], featurevector_allmoments[test_index]
        f_train, f_test = labels_allmoments[train_index], labels_allmoments[test_index]
        clf = RandomForestClassifier(**best_params)
        clf.fit(X_train, f_train)
        f_predicted = clf.predict(X_test)
        tp[i], tn[i], fp[i], fn[i], acc[i], tss[i] = outputclass_analysis_scorereturn(f_test, f_predicted)

    print("TP = " + str(np.mean(tp)) + "+/-" + str(np.std(tp)))
    print("TN = " + str(np.mean(tn)) + "+/-" + str(np.std(tn)))
    print("FP = " + str(np.mean(fp)) + "+/-" + str(np.std(fp)))
    print("FN = " + str(np.mean(fn)) + "+/-" + str(np.std(fn)))
    print("Acc = " + str(np.mean(acc)) + "+/-" + str(np.std(acc)))
    print("TSS = " + str(np.mean(tss)) + "+/-" + str(np.std(tss)))
    

print("-------------------------------------")
print("Testing for threshold THRES = 0.0001:")
evaluate_RFclassifier(featurevector_allmoments, labels_allmoments_001, data_split)

print("-------------------------------------")
print("Testing for threshold THRES = 0.0005:")
evaluate_RFclassifier(featurevector_allmoments, labels_allmoments_005, data_split)

print("-------------------------------------")
print("Testing for threshold THRES = 0.001:")
evaluate_RFclassifier(featurevector_allmoments, labels_allmoments_01, data_split)

print("-------------------------------------")
print("Testing for threshold THRES = 0.005:")
evaluate_RFclassifier(featurevector_allmoments, labels_allmoments_05, data_split)

print("-------------------------------------")
print("Testing for threshold THRES = 0.01:")
evaluate_RFclassifier(featurevector_allmoments, labels_allmoments_10, data_split)

-------------------------------------
Testing for threshold THRES = 0.0001:
Fitting 10 folds for each of 40 candidates, totalling 400 fits
Best parameters found:  {'class_weight': 'balanced', 'max_depth': 10, 'n_estimators': 200}
TP = 435.5+/-7.0178344238090995
TN = 69.7+/-5.273518749374083
FP = 9.6+/-3.4698703145794947
FN = 12.2+/-2.6758176320519302
Acc = 0.9586337760910816+/-0.00592808324546996
TSS = 0.8536900113303822+/-0.033144222483425786
-------------------------------------
Testing for threshold THRES = 0.0005:
Fitting 10 folds for each of 40 candidates, totalling 400 fits
Best parameters found:  {'class_weight': 'balanced', 'max_depth': 25, 'n_estimators': 50}
TP = 207.6+/-5.023942674832188
TN = 293.9+/-7.063285354564121
FP = 10.8+/-3.370459909270543
FN = 14.7+/-5.040833264451424
Acc = 0.9516129032258066+/-0.010436432637571148
TSS = 0.8989368471317333+/-0.022424705612163826
-------------------------------------
Testing for threshold THRES = 0.001:
Fitting 10 folds for each of 4

In [12]:
def tss_score(test_labels, predicted_labels):
    matrix_elements = confusion_matrix(test_labels, predicted_labels, labels=[0, 1]).ravel()
    tn, fp, fn, tp = matrix_elements
    tss = tp/(tp+fn) - fp/(fp+tn) if (((tp+fn) != 0) and ((fp+tn) != 0)) else -999.9
    return tss


def evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments, data_split, tss_scorer):

    # parameter grid
    param_grid = {'n_estimators': [10,50,100,200], \
                  'max_depth': [None, 2, 5, 10, 25], \
                  'class_weight': [None, 'balanced']}

    # classifier and gridsearch
    clf = RandomForestClassifier()
    grid_search = GridSearchCV(clf, param_grid, cv=data_split, verbose=1, scoring=tss_scorer)
    grid_search.fit(featurevector_allmoments, labels_allmoments)
    print("Best parameters found: ", grid_search.best_params_)
    best_params = grid_search.best_params_

    # fitting 10 times and accumulating the scores for the best model
    tp = np.zeros([10], dtype=int)
    tn = np.zeros([10], dtype=int)
    fp = np.zeros([10], dtype=int)
    fn = np.zeros([10], dtype=int)
    acc = np.zeros([10], dtype=float)
    tss = np.zeros([10], dtype=float)

    for i, split_indexes in enumerate(data_split.split(featurevector_allmoments)):
        train_index, test_index = split_indexes
        X_train, X_test = featurevector_allmoments[train_index], featurevector_allmoments[test_index]
        f_train, f_test = labels_allmoments[train_index], labels_allmoments[test_index]
        clf = RandomForestClassifier(**best_params)
        clf.fit(X_train, f_train)
        f_predicted = clf.predict(X_test)
        tp[i], tn[i], fp[i], fn[i], acc[i], tss[i] = outputclass_analysis_scorereturn(f_test, f_predicted)

    print("TP = " + str(np.mean(tp)) + "+/-" + str(np.std(tp)))
    print("TN = " + str(np.mean(tn)) + "+/-" + str(np.std(tn)))
    print("FP = " + str(np.mean(fp)) + "+/-" + str(np.std(fp)))
    print("FN = " + str(np.mean(fn)) + "+/-" + str(np.std(fn)))
    print("Acc = " + str(np.mean(acc)) + "+/-" + str(np.std(acc)))
    print("TSS = " + str(np.mean(tss)) + "+/-" + str(np.std(tss)))
    

tss_scorer = make_scorer(tss_score, greater_is_better=True)
    
print("-------------------------------------")
print("Testing for threshold THRES = 0.0001:")
evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments_001, data_split, tss_scorer)

print("-------------------------------------")
print("Testing for threshold THRES = 0.0005:")
evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments_005, data_split, tss_scorer)

print("-------------------------------------")
print("Testing for threshold THRES = 0.001:")
evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments_01, data_split, tss_scorer)

print("-------------------------------------")
print("Testing for threshold THRES = 0.005:")
evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments_05, data_split, tss_scorer)

print("-------------------------------------")
print("Testing for threshold THRES = 0.01:")
evaluate_RFclassifier_score(featurevector_allmoments, labels_allmoments_10, data_split, tss_scorer)

-------------------------------------
Testing for threshold THRES = 0.0001:
Fitting 10 folds for each of 40 candidates, totalling 400 fits
Best parameters found:  {'class_weight': 'balanced', 'max_depth': 5, 'n_estimators': 100}
TP = 426.3+/-10.159232254457025
TN = 74.2+/-5.89576118919347
FP = 5.1+/-2.947880594596735
FN = 21.4+/-6.711184694225007
Acc = 0.9497153700189752+/-0.012904620976029823
TSS = 0.8894599427968715+/-0.03339451441033196
-------------------------------------
Testing for threshold THRES = 0.0005:
Fitting 10 folds for each of 40 candidates, totalling 400 fits
Best parameters found:  {'class_weight': None, 'max_depth': 25, 'n_estimators': 200}
TP = 207.3+/-5.1000000000000005
TN = 294.0+/-7.0
FP = 10.7+/-3.950949253027682
FN = 15.0+/-4.381780460041329
Acc = 0.9512333965844404+/-0.010114047796004891
TSS = 0.8978686577010769+/-0.020736718025196818
-------------------------------------
Testing for threshold THRES = 0.001:
Fitting 10 folds for each of 40 candidates, totallin